## **Python for Quantitative Trading: Futures**
### **CTA Strategies - Introduction**

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat July 25 15:28:01 2021

@author: Bradley

Slides for practicing quantitative trading
"""

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from datetime import datetime

plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签`
plt.rcParams['axes.unicode_minus'] = False

%config InlineBackend.figure_format = 'svg' #在notebook中可以更好的显示，svg输出是一种向量化格式，缩放网页并不会导致图片失真。这行代码似乎只用在ipynb文件中才能使用。

%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')  # 忽略一些warnings

# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('expand_frame_repr', False)  
pd.set_option("display.max_rows", 500) 
pd.set_option('display.max_columns', None) #防止column太多中间变成省略号

In [ ]:
# TwoLines.py
"""
一、工具包导入
"""
from atrader import *
import numpy as np
"""
二、初始化
"""
def init(context):
    # 注册数据
    reg_kdata('day', 1)                        # 注册日频行情数据
    # 回测细节设置
    set_backtest(initial_cash=1e8)             # 初始化设置账户总资金
    # 全局变量定义/参数定义
    context.win = 21                            # 计算所需总数据长度
    context.long_win = 20                       # 20日均线（长均线）参数
    context.short_win = 5                       # 5日均线（短均线）参数
    context.Tlen = len(context.target_list)     # 标的数量
"""
三、策略运行逻辑函数
"""
def on_data(context):
    # 获取注册数据
    # 所有标的的K线行情数据
    data = get_reg_kdata(reg_idx=context.reg_kdata[0], length=context.win, fill_up=True, df=True)
    if data['close'].isna().any():                                    # 行情数据若存在nan值，则跳过
        return
    close = data.close.values.reshape(-1, context.win).astype(float)   # 获取收盘价，并转为ndarray类型的二维数组
    # 仓位数据查询
    positions = context.account().positions['volume_long'].values    # 获取仓位数据：positions=0，表示无持仓
    # 逻辑计算
    mashort = close[:, -5:].mean(axis=1)                    # 短均线：5日均线
    malong = close[:, -20:].mean(axis=1)                    # 长均线：20日均线
    target = np.array(range(context.Tlen))                   # 获取标的序号
    long = np.logical_and(positions == 0, mashort > malong)     # 未持仓，且短均线上穿长均线为买入信号，逻辑同时成立
    short = np.logical_and(positions > 0, mashort < malong)     # 持仓，且短均线下穿长均线为卖出信号
    target_long = target[long].tolist()                      # 获取买入信号标的的序号
    target_short = target[short].tolist()                    # 获取卖出信号标的的序号
    # 策略下单交易：
    for targets in target_long:
        order_target_value(account_idx=0, target_idx=targets, target_value=1e8/context.Tlen, side=1,order_type=2, price=0) # 买入下单
    for targets in target_short:
        order_target_volume(account_idx=0, target_idx=targets, target_volume=0, side=1,order_type=2, price=0)              # 卖出平仓
"""
四、策略执行脚本
"""
if __name__ == '__main__':
    # 策略回测函数
    run_backtest(strategy_name='TwoLines', file_path='TwoLines.py', target_list=get_code_list('hs300')['code'],
                 frequency='day', fre_num=1, begin_date='2019-01-01', end_date='2019-08-01', fq=1)

In [ ]:
# BiasAverage.py
"""
一、工具包导入
"""
from atrader import *
import numpy as np
import pandas as pd

"""
二、初始化
"""
def init(context):
    # 注册数据
    reg_kdata('min', 5)                         # 注册日频行情数据
    # 回测细节设置
    set_backtest(initial_cash=1e8)             # 初始化设置账户总资金
    # 全局变量定义/参数定义
    context.initial = 1e8
    context.win = 300                            # 计算所需总数据长度
    context.long_win = 135                       # 慢速均线（长均线）参数
    context.short_win = 30                       # 快速均线（短均线）参数
    context.ema_win = 30                         # 乖离率长度
    context.n = 100                              # 离差滑动平均值阈值
    context.Tlen = len(context.target_list)      # 标的数量
"""
三、策略运行逻辑函数
"""
def on_data(context):
    # 获取注册数据
    data = get_reg_kdata(reg_idx=context.reg_kdata[0], length=context.win, fill_up=True, df=True)      # 所有标的的K线行情数据
    if data['close'].isna().any():                                    # 行情数据若存在nan值，则跳过
        return
    close = data.close.values.reshape(-1, context.win).astype(float)   # 获取收盘价，并转为ndarray类型的二维数组

    # 仓位数据查询
    long_positions = context.account().positions['volume_long']
    short_positions = context.account().positions['volume_short']    # 获取仓位数据：positions=0，表示无持仓

    # 循环判断并交易每个标的
    for i in range(context.Tlen):
        # 策略的逻辑计算
        mashort = get_EMA(close[i,-context.win:],context.short_win)        # 短均线：5日均线
        malong = get_EMA(close[i,-context.win:],context.long_win)          # 长均线：20日均线
        values = mashort - malong                                          # 均线差值
        bias = get_EMA(values,context.ema_win)                             # 均线乖离

        # 进出场交易条件的计算
        # 快速均线上穿慢速均线后，又掉头向下靠近慢速均线但不向下突破，再上拉时买入
        con1 = (values[-2] > 0) & (bias[-3] > bias[-2] )& (bias[-2] < bias[-1])& (np.abs(bias[-1]) < context.n)
        # 慢速均线上穿快速均线后，又掉头向下靠近快速均线但不向下突破，再上拉时卖出
        con2 = (values[-2] < 0) & (bias[-3] < bias[-2]) & (bias[-2] > bias[-1]) & (np.abs(bias[-1]) < context.n)
        con3 = mashort[-1] > malong[-1]     # 慢速均线上穿快速均线
        con4 = mashort[-1] < malong[-1]     # 快速均线上穿慢速均线

        # 下单交易
        # 无持仓
        if (long_positions[i] == 0) & (short_positions[i] == 0):
            if con1 & con3:                                                      # 多单进场
                order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                   side=1, order_type=2)
            elif con2 & con4:                                                    # 空单进场
                order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                   side=2, order_type=2)

        # 持有多单
        if con4 & (long_positions[i] > 0):                                        # 多单出场
            order_target_volume(account_idx=0, target_idx=i, target_volume=0, side=1, order_type=2)

        # 持有空单
        if con3 & (short_positions[i] > 0):                                       # 空单出场
            order_target_volume(account_idx=0, target_idx=i, target_volume=0, side=2, order_type=2)

"""
四、策略执行脚本
"""
if __name__ == '__main__':
    target_list = ['SHFE.rb0000', 'SHFE.hc0000']  # 设置回测标的

    # 策略回测函数
    run_backtest(strategy_name='BiasAverage', file_path='BiasAverage.py', target_list=target_list,
                 frequency='min', fre_num=5, begin_date='2016-01-01', end_date='2019-01-01', fq=1)

# EMA的计算函数定义法一
def get_EMA(df,length):
    k = 2/(length+1)
    df2 = [0]*len(df)     # 数组形式
    for i in range(len(df)):
        if i==0:
            df2[i] = df[i]
        if i>0:
            df2[i] = k * df[i] + (1-k) * df2[i-1]
    return np.asarray(df2)

## EMA的计算函数定义法二
# def get_EMA(df,length):
#     k = 2/(length+1)
#     df2 = []
#     for i in range(len(df)):
#         if i==0:
#             df2.append(df[i])
#         if i>0:
#             df2.append(k * df[i] + (1-k) * df2[i-1])
#     return np.asarray(df2)

In [ ]:
# DualThrust.py
"""
一、工具包导入
"""
from atrader import *
import numpy as np
import pandas as pd

"""
二、初始化
"""
def init(context):
    # 注册数据
    reg_kdata('day', 1)                         # 注册日频行情数据
    # 回测细节设置
    set_backtest(initial_cash=1e8)             # 初始化设置账户总资金
    # 全局变量定义/参数定义
    context.initial = 1e8
    context.win = 2                             # 计算所需总数据长度
    context.k1 = 0.4                            # 上轨参数参数
    context.k2 = 0.4                            # 下轨参数参数
    context.n = 1                               # 天数参数
    context.Tlen = len(context.target_list)     # 标的数量
"""
三、策略运行逻辑函数
"""
def on_data(context):
    # 获取注册数据
    data = get_reg_kdata(reg_idx=context.reg_kdata[0], length=context.win, fill_up=True, df=True)  # 所有标的的K线行情数据
    if data['close'].isna().any():               # 行情数据若存在nan值，则跳过
        return
    # 仓位数据查询
    long_positions = context.account().positions['volume_long']      # 获取多头仓位数据
    short_positions = context.account().positions['volume_short']    # 获取空头仓位数据
    # 时间计算
    bar_info = get_current_bar([0])               # 获取当前bar的信息
    bartime = bar_info.iloc[0, 1]
    nowtime = bartime.hour * 100 + bartime.minute
    close = data.close.values.reshape(-1, context.win).astype(float)   # 获取收盘价，并转为ndarray类型的二维数组
    high = data.high.values.reshape(-1, context.win).astype(float)     # 获取最高价，并转为ndarray类型的二维数组
    low = data.low.values.reshape(-1, context.win).astype(float)       # 获取最低价，并转为ndarray类型的二维数组
    open = data.open.values.reshape(-1, context.win).astype(float)     # 获取开盘价，并转为ndarray类型的二维数组

    if (nowtime >= 1455) & (nowtime < 1500):
        if (long_positions.any() != 0)|(long_positions.any() != 0):
            order_close_all()  # 日内平仓
            return
    # print(nowtime)

    # 逻辑计算
    for i in range(context.Tlen):
        # 逻辑计算
        HH = high[i, -context.n-1:-1].max()  # 前n日的最高价
        HC = close[i, -context.n-1:-1].max()
        LC = close[i, -context.n-1:-1].min()
        LL = low[i, -context.n-1:-1].min()
        Range = max(HH-LC,HC-LL)                   # 波动范围

        # 计算通道的上下轨
        Buyline = open[i,-1] + context.k1 * Range
        Sellline = open[i,-1] - context.k2 * Range

        # 下单交易
        # 无持仓
        if (nowtime < 1455) | (nowtime > 1500):
            if (long_positions[i] == 0) & (short_positions[i] == 0):
                if close[i,-1] > Buyline:
                    order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                       side=1, order_type=2)
                elif close[i,-1] < Sellline:
                    order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                       side=2, order_type=2)
            # 持多仓
            elif (long_positions[i] > 0) & (close[i,-1] < Sellline):
                order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                   side=2, order_type=2)
            # 持空仓
            elif (short_positions[i] > 0) & (close[i,-1] > Buyline):
                order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                   side=1, order_type=2)

"""
四、策略执行脚本
"""
if __name__ == '__main__':
    target_list = ['SHFE.rb0000', 'SHFE.hc0000']  # 设置回测标的

    # 策略回测函数
    run_backtest(strategy_name='DualThrust', file_path='DualThrust.py', target_list=target_list,
                 frequency='min', fre_num=1, begin_date='2016-01-01', end_date='2019-01-01', fq=1)



In [ ]:
# R_breaker.py
"""
一、工具包导入
"""
from atrader import *
import numpy as np
import pandas as pd

"""
二、初始化
"""
def init(context):
    # 注册数据
    reg_kdata('day', 1)                         # 注册日频行情数据
    # 回测细节设置
    set_backtest(initial_cash=1e8)              # 初始化设置账户总资金
    # 全局变量定义/参数定义
    context.initial = 1e8
    context.win = 2                              # 计算所需总数据长度
    context.stoploss = 0.005                      # 止损
    context.stopprofit = 0.005                    # 止盈
    context.Tlen = len(context.target_list)      # 标的数量
    context.record_entryP = np.array(np.zeros(context.Tlen))   # 记录入场点位
    context.record_reverse = np.array(np.zeros(context.Tlen))  # 记录反转点位

"""
三、策略运行逻辑函数
"""
def on_data(context):
    # 获取注册数据
    data = get_reg_kdata(reg_idx=context.reg_kdata[0], length=context.win, fill_up=True, df=True)  # 所有标的的K线行情数据
    if data['close'].isna().any():  # 行情数据若存在nan值，则跳过
        return
    # 仓位数据查询
    long_positions = context.account().positions['volume_long']    # 获取多头仓位数据
    short_positions = context.account().positions['volume_short']  # 获取空头仓位数据
    # 时间计算
    bar_info = get_current_bar([0])  # 获取当前bar的信息
    bartime = bar_info.iloc[0, 1]
    nowtime = bartime.hour * 100 + bartime.minute
    close = data.close.values.reshape(-1, context.win).astype(float)   # 获取收盘价，并转为ndarray类型的二维数组
    high = data.high.values.reshape(-1, context.win).astype(float)     # 获取最高价，并转为ndarray类型的二维数组
    low = data.low.values.reshape(-1, context.win).astype(float)       # 获取最低价，并转为ndarray类型的二维数组

    # 逻辑计算
    for i in range(context.Tlen):

        # 日内平仓
        if (nowtime >= 1455) & (nowtime < 1500):
            context.record_reverse[i] = 0  # 每日反转标记归0
            if (long_positions.any() != 0) | (long_positions.any() != 0):
                order_close_all()  # 日内平仓
                return

        # 逻辑计算，计算六个价位
        Ssetup = high[i,-2] + 0.35 * (close[i,-2] - low[i,-2])
        Bsetup = low[i,-2] - 0.35 * (high[i,-2] - close[i,-2])
        Senter = (1.07 / 2) * (high[i,-2] + low[i,-2]) - 0.07 * low[i,-2]
        Benter = (1.07 / 2) * (high[i,-2] + low[i,-2]) - 0.07 * high[i,-2]
        Sbreak = Bsetup - 0.25 * (Ssetup - Bsetup)
        Bbreak = Ssetup + 0.25 * (Ssetup - Bsetup)

        # 无持仓—进场
        if (nowtime < 1455) | (nowtime > 1500):
            if (long_positions[i] == 0) & (short_positions[i] == 0):    # 无持仓
                if close[i,-1] > Bbreak:
                    order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                       side=1, order_type=2)
                    context.record_entryP[i] = close[i,-1]             # 记录进场价格
                elif close[i,-1] < Sbreak:
                    order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                       side=2, order_type=2)
                    context.record_entryP[i] = close[i, -1]            # 记录进场价格

            # 持多仓—反转进场
            elif (long_positions[i] > 0):     # 持有多单
                # 止盈止损
                if close[i, -1] < context.record_entryP[i] * (1 - context.stopprofit):       # 止损
                    order_target_value(account_idx=0, target_idx=i, target_value=0, side=1, order_type=2)
                    context.record_entryP[i] = 0                           # 出场后价格记录为0
                elif close[i, -1] > context.record_entryP[i] * (1 + context.stoploss):       # 止盈
                    order_target_value(account_idx=0, target_idx=i, target_value=0, side=1, order_type=2)
                    context.record_entryP[i] = 0
                # 反转标记
                if high[i,-1] > Ssetup:                                           # 当日内最高价高于观察价,标记为1
                    context.record_reverse[i] = 1
                # 反转进场
                if (close[i,-1] < Senter) & (context.record_reverse[i] == 1):
                    order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                       side=2, order_type=2)
                    context.record_reverse[i] = 0                          # 进场后标记回归为0
                    context.record_entryP[i] = close[i, -1]                # 记录反转进场价格

            # 持空仓—反转进场
            elif (short_positions[i] > 0):     # 持有空单
                # 止盈止损
                if close[i,-1] < context.record_entryP[i] * (1 - context.stopprofit):         # 止盈
                    order_target_value(account_idx=0,target_idx=i,target_value=0,side=2, order_type=2)
                    context.record_entryP[i] = 0
                elif close[i,-1] > context.record_entryP[i] * (1 + context.stoploss):         # 止损
                    order_target_value(account_idx=0, target_idx=i, target_value=0, side=2, order_type=2)
                    context.record_entryP[i] = 0
                # 反转标记
                if low[i,-1] < Bsetup:                                          # 当日内最低价低于观察价,标记为2
                    context.record_reverse[i] = 2
                # 符合反转则进场
                if (close[i,-1] > Benter) & (context.record_reverse[i] == 2):
                    order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                       side=1, order_type=2)
                    context.record_reverse[i] = 0                                # 进场后标记回归为0
                    context.record_entryP[i] = close[i, -1]                      # 记录反转进场价格

"""
四、策略执行脚本
"""
if __name__ == '__main__':
    target_list = ['SHFE.rb0000', 'SHFE.hc0000']  # 设置回测标的

    # 策略回测函数
    run_backtest(strategy_name='R_breaker', file_path='R_breaker.py', target_list=target_list,
                 frequency='min', fre_num=5, begin_date='2016-01-01', end_date='2019-01-01', fq=1)



In [ ]:
# BollBreak.py
"""
一、工具包导入
"""
from atrader import *
import numpy as np
import pandas as pd

"""
二、初始化
"""
def init(context):
    # 注册数据
    reg_kdata('day', 1)                           # 注册日频行情数据
    # 回测细节设置
    set_backtest(initial_cash=1e8)                # 初始化设置账户总资金
    # 全局变量定义/参数定义
    context.initial = 1e8
    context.win = 21                               # 计算所需总数据长度
    context.len = 20                               # 布林带/ATR长度
    context.K = 1.4                                # 布林带上下轨参数
    context.N = 0.4                                # 止损ATR倍数
    context.M = 0.4                                # 止盈ATR倍数
    context.Tlen = len(context.target_list)        # 标的数量
    context.record_entryP = np.array(np.zeros(context.Tlen))     # 记录入场点位
"""
三、策略运行逻辑函数
"""
def on_data(context):
    # 获取注册数据
    data = get_reg_kdata(reg_idx=context.reg_kdata[0], length=context.win, fill_up=True, df=True)  # 所有标的的K线行情数据
    if data['close'].isna().any():  # 行情数据若存在nan值，则跳过
        return
    # 仓位数据查询
    long_positions = context.account().positions['volume_long']    # 获取多头仓位数据
    short_positions = context.account().positions['volume_short']  # 获取空头仓位数据
    # 数据计算
    close = data.close.values.reshape(-1, context.win).astype(float)   # 获取收盘价，并转为ndarray类型的二维数组
    high = data.high.values.reshape(-1, context.win).astype(float)     # 获取最高价，并转为ndarray类型的二维数组
    low = data.low.values.reshape(-1, context.win).astype(float)       # 获取最低价，并转为ndarray类型的二维数组

    # 逻辑计算
    for i in range(context.Tlen):

        # 布林带计算
        MiddleLine = close[i,-context.len:].mean()                          # 均线
        UpperLine = MiddleLine + context.K * close[i,-context.len:].std()   # 布林带上轨
        LowerLine = MiddleLine - context.K * close[i,-context.len:].std()   # 布林带下轨

        # ATR计算
        HL = high[i,-2] - low[i,-2]
        HC = abs(high[i,-2] - close[i,-3])
        CL = abs(low[i,-2] - close[i,-3])
        ATR = max(max(HL, HC), CL)        # TR真实波幅

        # 无持仓—进场
        if (long_positions[i] == 0) & (short_positions[i] == 0):    # 无持仓
            if close[i,-1] > UpperLine:
                order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                   side=1, order_type=2)
                context.record_entryP[i] = close[i,-1]             # 记录进场价格
            elif close[i,-1] < LowerLine:
                order_target_value(account_idx=0, target_idx=i, target_value=context.initial / context.Tlen,
                                   side=2, order_type=2)
                context.record_entryP[i] = close[i, -1]            # 记录进场价格

        # 持多仓—出场
        elif (long_positions[i] > 0):     # 持有多单
            # 止盈止损
            if (close[i, -1] < context.record_entryP[i] - context.N * ATR)|(close[i, -1] > (context.record_entryP[i] +  context.M * ATR)):          # 多单止损/止盈
                order_target_value(account_idx=0, target_idx=i, target_value=0, side=1, order_type=2)
                context.record_entryP[i] = 0

        # 持空仓—出场
        elif (short_positions[i] > 0):     # 持有空单
            # 止盈止损
            if (close[i,-1] < (context.record_entryP[i] - context.M * ATR))|(close[i,-1] > (context.record_entryP[i] +  context.N * ATR)):       # 空单止盈/止损
                order_target_value(account_idx=0,target_idx=i,target_value=0,side=2, order_type=2)
                context.record_entryP[i] = 0

"""
四、策略执行脚本
"""
if __name__ == '__main__':
    target_list = ['SHFE.rb0000', 'SHFE.hc0000']  # 设置回测标的

    # 策略回测函数
    run_backtest(strategy_name='BollBreak', file_path='BollBreak.py', target_list=target_list,
                 frequency='day', fre_num=1, begin_date='2016-01-01', end_date='2019-01-01', fq=1)

In [ ]:
# Turtle.py
"""
一、工具包导入
"""
from atrader import *
import numpy as np
import pandas as pd

"""
二、初始化
"""
def init(context):
    # 注册数据
    reg_kdata('day', 1)                           # 注册日频行情数据
    # 回测细节设置
    set_backtest(initial_cash=1e8, margin_rate=1.0, slide_price=0.0,
                 price_loc=1, deal_type=0, limit_type=0)
    # 全局变量定义/参数定义
    context.initial = 1e8                          # 初始化设置账户总资金
    context.win = 22                               # 计算所需总数据长度，需比进场数据长度大2
    context.system_in = 20                         # 系统的进场数据长度
    context.system_out = 10                        # 系统的出场数据长度
    context.ATR_N = 20                             # ATR的数据长度
    context.add = 0.5                              # 加仓参数
    context.stop_loss = 2                          # 止损参数
    context.Tlen = len(context.target_list)        # 标的数量
    context.record_entryP = np.array(np.zeros(context.Tlen))     # 记录入场点位
    context.future_info = get_future_info(context.target_list)   # 获取期货标的基本信息

"""
三、策略运行逻辑函数
"""
def on_data(context):
    # 获取注册数据
    data = get_reg_kdata(reg_idx=context.reg_kdata[0], length=context.win, fill_up=True, df=True)
    if data['close'].isna().any():  # 行情数据若存在nan值，则跳过
        return
    # 仓位数据查询
    long_positions = context.account().positions['volume_long']    # 获取多头仓位数据
    short_positions = context.account().positions['volume_short']  # 获取空头仓位数据
    # 数据计算
    close = data.close.values.reshape(-1, context.win).astype(float)   # 获取收盘价，并转为ndarray类型的二维数组
    high = data.high.values.reshape(-1, context.win).astype(float)     # 获取最高价，并转为ndarray类型的二维数组
    low = data.low.values.reshape(-1, context.win).astype(float)       # 获取最低价，并转为ndarray类型的二维数组

    # 逻辑计算
    for i in range(context.Tlen):

        # 计算系统进出场唐奇安通道
        system_in_up = max(high[i,-context.system_in-1:-1])                   # 系统的进场的上轨
        system_in_down = min(low[i, -context.system_in-1:-1])                # 系统的进场的下轨
        system_out_up = max(high[i,-context.system_out-1:-1])                 # 系统的出场的上轨
        system_out_down = min(low[i, -context.system_out-1:-1])              # 系统的出场的下轨

        # ATR计算
        HL = (high[i,-context.ATR_N-1:-1] - low[i,-context.ATR_N-1:-1])          # 当前交易日的最高价与最低价间的波幅
        HC = abs(high[i,-context.ATR_N-1:-1] - close[i,-context.ATR_N-2:-2])     # 前一交易日收盘价与当个交易日最高价间的波幅
        CL = abs(low[i,-context.ATR_N-1:-1] - close[i,-context.ATR_N-2:-2])      # 前一交易日收盘价与当个交易日最低价间的波幅
        TR = np.max([HL,HC,CL],axis = 0)                                         # n日的真实波幅
        ATR = TR.mean()                                                          # n日的真实波幅的均值

        # 每次要买入的仓位
        min_move = context.future_info['min_move'][i]                           # 标的最小变动单位
        Unit = context.initial * 0.01 / ATR / min_move / context.Tlen           # 标的每次买入仓位

        # 无持仓—进场
        if (long_positions[i] == 0) & (short_positions[i] == 0):   # 无持仓
            if close[i,-1] > system_in_up:                                                # 进多单
                order_target_volume(account_idx=0, target_idx=i, target_volume=int(Unit), side=1, order_type=2)
                context.record_entryP[i] = close[i,-1]             # 记录进场价格

            elif close[i,-1] < system_in_down:                                            # 进空单
                order_target_volume(account_idx=0, target_idx=i, target_volume=int(Unit), side=2, order_type=2)
                context.record_entryP[i] = close[i, -1]            # 记录进场价格

        # 持多仓—加仓/出场
        elif (long_positions[i] > 0):     # 持有多单
            if close[i, -1] > context.record_entryP[i] + context.add * ATR:                # 多单加仓
                order_target_volume(account_idx=0, target_idx=i, target_volume=int(Unit), side=1, order_type=2)
                context.record_entryP[i] = close[i, -1]            # 记录进场价格

            elif close[i, -1] < system_out_down:                                           # 多单离市
                order_target_volume(account_idx=0, target_idx=i, target_volume=0, side=1, order_type=2)
                context.record_entryP[i] = 0

            elif close[i, -1] < context.record_entryP[i] - context.stop_loss * ATR:        # 多单止损
                order_target_volume(account_idx=0, target_idx=i, target_volume=0, side=1, order_type=2)
                context.record_entryP[i] = 0

        # 持空仓—加仓/出场
        elif (short_positions[i] > 0):     # 持有空单
            if close[i,-1] <  context.record_entryP[i] - context.add * ATR :               # 空单加仓
                order_target_volume(account_idx=0,target_idx=i,target_volume=int(Unit),side=2, order_type=2)
                context.record_entryP[i] = close[i, -1]            # 记录进场价格

            elif close[i, -1] > system_out_up:                                             # 空单离市
                order_target_volume(account_idx=0, target_idx=i, target_volume=0, side=2, order_type=2)
                context.record_entryP[i] = 0

            elif close[i,-1] > context.record_entryP[i] + context.stop_loss * ATR:         # 空单止损
                order_target_volume(account_idx=0, target_idx=i, target_volume=0, side=2, order_type=2)
                context.record_entryP[i] = 0

"""
四、策略执行脚本
"""
if __name__ == '__main__':
    target_list = ['SHFE.zn0000', 'SHFE.rb0000']                    # 设置回测标的
    # 策略回测函数
    run_backtest(strategy_name='Turtle', file_path='Turtle.py', target_list=target_list,
                 frequency='min', fre_num=5, begin_date='2016-01-01', end_date='2019-01-01', fq=1)

In [ ]:
# ADFtest.py
"""
一、工具包导入
"""
from atrader import *
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

target_list = ['cffex.IF0000', 'cffex.IF0001']  #设定标的
data=get_kdata(target_list,'day',1, begin_date='2016-01-01', end_date='2019-01-01', fill_up=True, df=True)  #获取标的历史数据
close=data.close.values.reshape(2, -1).astype(float)             #提取收盘价
times=data.time.values.reshape(2,-1)                             #提取日期
IF0000=close[0,:]                      #IF主力合约价格
IF0001=close[1,:]                      #IF次主力合约价格
a=np.corrcoef(IF0000, IF0001)            #求相关系数
X=sm.add_constant(IF0001)              #加入常数项
result=sm.OLS(IF0000, X).fit()          #回归
#print (result.summary())
result2=sm.tsa.stattools.adfuller(result.resid)        #对残差进行ADF检验
print (result2)
IF=result.predict(X)                   #回归预测值

#画图
plt.figure(figsize = (8,6.4))
plt.scatter(IF0001, IF0000)                #画出回归的散点图和回归线
plt.plot(IF0001,IF,c='r')
#plt.plot(result.resid)                    #画回归残差图
#plt.xlabel('case number')
#plt.ylabel('resid')
#plt.plot(times[0,1:],np.diff(IF0000),label='IF0000')            #画一阶差分数据图
#plt.plot(times[0,1:], np.diff(IF0001),label='IF0001')
plt.show()


In [ ]:
# StraddleIF.py
"""
一、工具包导入
"""
from atrader import *
import numpy as np
import pandas as pd

"""
二、初始化
"""
def init(context):
    # 注册数据
    reg_kdata('day', 1)                           # 注册日频行情数据
    # 回测细节设置
    set_backtest(initial_cash=1e8)               # 初始化设置账户总资金
    # 全局变量定义/参数定义
    context.initial = 1e8
    context.win = 60                               # 计算所需总数据长度
    context.N = 60                                 # N日的波动范围
    context.K = 1.3                                # 进场上下轨参数
    context.P = 0.3                                # 出场上下轨参数
    context.Tlen = len(context.target_list)        # 标的数量

"""
三、策略运行逻辑函数
"""
def on_data(context):
    # 获取注册数据
    data = get_reg_kdata(reg_idx=context.reg_kdata[0], length=context.win, fill_up=True, df=True)  # 所有标的的K线行情数据
    if data['close'].isna().any():  # 行情数据若存在nan值，则跳过
        return
    # 仓位数据查询
    long_positions = context.account().positions['volume_long']    # 获取多头仓位数据
    short_positions = context.account().positions['volume_short']  # 获取空头仓位数据
    # 数据计算
    close = data.close.values.reshape(-1, context.win).astype(float)   # 获取收盘价，并转为ndarray类型的二维数组

    # 逻辑计算
    spread = close[0,:] - close[1,:]     # 价差
    ma = spread.mean()                   # 均值
    st = spread.std()                    # 标准差

    # 计算上下轨
    UpLine = ma + context.K * st          # 上轨
    DownLine = ma - context.K * st        # 下轨

    # 无持仓—进场
    if (long_positions[0] == 0) & (short_positions[0] == 0) & (long_positions[1] == 0) & (short_positions[1] == 0):    # 无持仓
        if spread[-1] < DownLine:                                   # 小于下轨，开价差多单
            order_target_volume(account_idx=0, target_idx=0, target_volume=100, side=1, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=100,side=2, order_type=2)

        elif spread[-1] > UpLine:                                  # 大于上轨，开价差空单
            order_target_volume(account_idx=0, target_idx=0, target_volume=100, side=2, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=100, side=1, order_type=2)


    # 持有仓位
    elif (long_positions[0] > 0) & (short_positions[1] > 0):        # 价差多单
        if spread[-1] > (ma + context.P * st):                                       # 止盈
            order_target_volume(account_idx=0, target_idx=0, target_volume=0, side=1, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=0, side=2, order_type=2)

    elif (short_positions[0] > 0) & (long_positions[1] > 0):        # 价差空单
        if spread[-1] < (ma - context.P * st):                                       # 止盈
            order_target_volume(account_idx=0, target_idx=0, target_volume=0, side=2, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=0, side=1, order_type=2)

    # 只有一个品种持有仓位
    elif (long_positions[0] == 0 & short_positions[1] != 0) | (long_positions[0] != 0 & short_positions[1] == 0):
        order_target_volume(account_idx=0, target_idx=0, target_volume=0, side=2, order_type=2)
        order_target_volume(account_idx=0, target_idx=1, target_volume=0, side=1, order_type=2)

"""
四、策略执行脚本
"""
if __name__ == '__main__':
    target_list = ['cffex.IF0000', 'cffex.IF0001']  # 设置回测标的

    # 策略回测函数
    run_backtest(strategy_name='StraddleIF', file_path='StraddleIF.py', target_list=target_list,
                 frequency='min', fre_num=15, begin_date='2016-01-01', end_date='2019-01-01', fq=1)



In [ ]:
# ADFtestJ.py

#工具包导入

from atrader import *
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

target_list = ['DCE.J0000', 'DCE.JM0000']              #设定标的
data=get_kdata(target_list,'day',1, begin_date='2016-01-01', end_date='2019-01-01', fill_up=True, df=True)  #获取标的历史数据
close=data.close.values.reshape(2, -1).astype(float)             #提取收盘价
times=data.time.values.reshape(2,-1)                             #提取日期
J0000=close[0,:]                      #焦炭合约价格
JM0000=close[1,:]                     #焦煤合约价格
a=np.corrcoef(J0000, JM0000)            #求相关系数
X=sm.add_constant(JM0000)              #加入常数项
result=sm.OLS(J0000, X).fit()          #回归
#print (result.summary())
result2=sm.tsa.stattools.adfuller(result.resid)        #对残差进行ADF检验
print (result2)
J01=result.predict(X)                   #回归预测值

#画图
#plt.plot(times[0,1:],np.diff(J0000),label='J0000')            #画一阶差分数据图
#plt.plot(times[0,1:], np.diff(JM0000),label='JM0001')
plt.figure(figsize = (8,6.4))                  #画回归散点图与回归曲线
plt.scatter(JM0000, J0000)
plt.plot(JM0000,J01,c='r')
#plt.plot(result.resid)                      #画回归残差图
#plt.xlabel('case number')
#plt.ylabel('resid')
plt.show()

spread = J0000 - 1.8 * JM0000
ma = spread.mean()
st = spread.std()
r=(spread-ma)/st
plt.figure()
plt.hist(spread)       #分布频率直方图
plt.show()


In [ ]:
# StraddleJ.py
"""
一、工具包导入
"""
from atrader import *
import numpy as np
import pandas as pd

"""
二、初始化
"""
def init(context):
    # 注册数据
    reg_kdata('day', 1)                           # 注册日频行情数据
    # 回测细节设置
    set_backtest(initial_cash=1e8)               # 初始化设置账户总资金
    # 全局变量定义/参数定义
    context.initial = 1e8
    context.win = 160                              # 计算所需总数据长度
    context.n = 10                                 # N 日的波动范围
    context.Nf = 4                                 # 短周期均线
    context.Ns = 60                                # 长周期均线
    context.k = 0.8                                # 上下轨参数
    context.P = 0.2                                # 上下轨止盈参数
    context.R = 1.5                                # AMA 止损参数
    context.l = 0                                  #交易次数计数
    context.profit = 0
    context.error = 0
    context.reduce = 0
    context.Tlen = len(context.target_list)        # 标的数量
    context.record_entryP = np.array(np.zeros(context.Tlen))     # 记录入场点位

"""
三、策略运行逻辑函数
"""
def on_data(context):
    # 获取注册数据
    data = get_reg_kdata(reg_idx=context.reg_kdata[0], length=context.win, fill_up=True, df=True)  # 所有标的的K线行情数据
    if data['close'].isna().any():  # 行情数据若存在nan值，则跳过
        return
    # 仓位数据查询
    long_positions = context.account().positions['volume_long']    # 获取多头仓位数据
    short_positions = context.account().positions['volume_short']  # 获取空头仓位数据
    # 数据计算
    close = data.close.values.reshape(-1, context.win).astype(float)   # 获取收盘价，并转为ndarray类型的二维数组
    spread = close[0,:] - 1.8*close[1,:]     # 价差

    # 自适应移动平均线AMA
    direction = abs(spread[-1] - spread[-context.n])     # 净价格变化
    vola = abs(np.subtract(spread[1:], spread[0:-1]))
    volatility = vola.sum()
    ER = direction / volatility
    fastest = 2 / ( context.Nf + 1)               # 短周期（快趋势）的平滑系数, Nf为短周期天数, 取4日
    slowest = 2 / ( context.Ns + 1)               # 长周期（慢趋势）的平滑系数，Ns为长周期天数，取60日
    smooth = ER * (fastest - slowest) + slowest
    c = smooth * smooth
    AMA = get_AMA(spread,c)

    # 价差正常波动范围
    ma = spread.mean()
    st = spread.std()
    up = ma + context.k * st
    down = ma - context.k * st

    # 无持仓—进场
    if (long_positions[0] == 0) & (short_positions[0] == 0) & (long_positions[1] == 0) & (short_positions[1] == 0):    # 无持仓
        if (spread[-1] < down) & (AMA[-2] > AMA[-3]) & (AMA[-3] < AMA[-4]):                                # 小于下轨，开价差多单
            order_target_volume(account_idx=0, target_idx=0, target_volume=1800, side=1, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=600, side=2, order_type=2)
            context.record_entryP = spread[-1]

        elif (spread[-1] > up) & (AMA[-2] < AMA[-3]) & (AMA[-3] > AMA[-4]):                               # 大于上轨，开价差空单
            order_target_volume(account_idx=0, target_idx=0, target_volume=1800, side=2, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=600, side=1, order_type=2)
            context.record_entryP = spread[-1]

    # 持有仓位
    elif (long_positions[0] > 0) & (short_positions[1] > 0):        # 价差多单
        if spread[-1] > (ma + context.P * st):                                       # 止盈
            order_target_volume(account_idx=0, target_idx=0, target_volume=0, side=1, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=0, side=2, order_type=2)
            context.profit = context.profit + 1
            print('profit', context.profit)

        if (spread[-1] < down) & (AMA[-2] < AMA[-3]) & (AMA[-3] > AMA[-4]):          # 减仓
            order_target_volume(account_idx=0, target_idx=0, target_volume=900, side=1, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=300, side=2, order_type=2)
            context.reduce = context.reduce + 1
            print('reduce', context.reduce)

        if AMA[-1] < context.record_entryP * context.R:                              # 止损
            order_target_volume(account_idx=0, target_idx=0, target_volume=0, side=1, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=0, side=2, order_type=2)
            context.l = context.l + 1
            print('l', context.l)

    elif (short_positions[0] > 0) & (long_positions[1] > 0):        # 价差空单
        if spread[-1] < (ma - context.P * st):                                       # 止盈
            order_target_volume(account_idx=0, target_idx=0, target_volume=0, side=2, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=0, side=1, order_type=2)
            context.profit = context.profit + 1
            print('profit', context.profit)

        if (spread[-1] > up) & (AMA[-2] > AMA[-3]) & (AMA[-3] < AMA[-4]):           # 减仓
            order_target_volume(account_idx=0, target_idx=0, target_volume=900, side=2, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=300, side=1, order_type=2)
            context.reduce = context.reduce + 1
            print('reduce', context.reduce)

        if AMA[-1] > context.record_entryP * context.R:                              # 止损
            order_target_volume(account_idx=0, target_idx=0, target_volume=0, side=2, order_type=2)
            order_target_volume(account_idx=0, target_idx=1, target_volume=0, side=1, order_type=2)
            context.l = context.l + 1
            print('l', context.l)

    # 只有一个品种持有仓位
    elif (long_positions[0] == 0 & short_positions[1] != 0) | (long_positions[0] != 0 & short_positions[1] == 0)|(long_positions[0] > 0 & long_positions[1] > 0)|(short_positions[0] > 0 & short_positions[1] > 0):
        order_target_volume(account_idx=0, target_idx=0, target_volume=0, side=2, order_type=2)
        order_target_volume(account_idx=0, target_idx=1, target_volume=0, side=1, order_type=2)
        order_target_volume(account_idx=0, target_idx=0, target_volume=0, side=1, order_type=2)
        order_target_volume(account_idx=0, target_idx=1, target_volume=0, side=2, order_type=2)
        context.error = context.error + 1
        print('error',context.error)

"""
四、策略执行脚本
"""
if __name__ == '__main__':
    target_list = ['DCE.J0000', 'DCE.JM0000']  # 设置回测标的

    # 策略回测函数
    run_backtest(strategy_name='StraddleJ', file_path='StraddleJ.py', target_list=target_list,
                 frequency='min', fre_num=60, begin_date='2016-01-01', end_date='2019-01-01', fq=1)

# AMA的计算函数定义法一
def get_AMA(df,constant):
    df2 = [0]*len(df)     # 数组形式
    for i in range(len(df)):
        if i==0:
            df2[i] = df[i]
        if i>0:
            df2[i] = df[i-1] + (df[i] - df2[i-1]) * constant
    return np.asarray(df2)